# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [24]:
# download necessary NLTK data
import nltk
nltk.download(['punkt', 'wordnet','stopwords'])

# import statements
import pandas as pd
import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sqlalchemy import create_engine 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from workspace_utils import active_session
 
from sklearn.linear_model import SGDClassifier

In [3]:
from sklearn.model_selection import GridSearchCV
import pickle

In [4]:
# load data from database
engine = create_engine('sqlite:///emergency.db')
df = pd.read_sql_table('emergency',engine)
df


,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,14,Information about the National Palace-,Informtion au nivaux palais nationl,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,15,Storm at sacred heart of jesus,Cyclone Coeur sacr de jesus,direct,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,16,"Please, we need tents and water. We are in Sil...",Tanpri nou bezwen tant avek dlo nou zon silo m...,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
8,17,"I would like to receive the messages, thank you",Mwen ta renmen jouin messag yo. Merci,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,18,I am in Croix-des-Bouquets. We have health iss...,"Nou kwadebouke, nou gen pwoblem sant m yo nan ...",direct,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1


In [5]:
X = df.message
y=df[df.columns[4:]]

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    url_regex = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

    # Detect URLs
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, 'urlplaceholder')

    # case normalization, punctuation removal and tokenization
    words = word_tokenize(re.sub(r'[^a-zA-Z0-9]', " ", text.lower()))

    # removing stop words
    words = [w for w in words if w not in stopwords.words("english")]

    # lemmatization
    lemmed = [WordNetLemmatizer().lemmatize(w, pos='v') for w in words]

    # stemming
    stemmed = [PorterStemmer().stem(w) for w in lemmed]
    return stemmed

In [7]:
for message in X[:5]:
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'updat', 'cold', 'front', 'cuba', 'could', 'pass', 'haiti'] 

Is the Hurricane over or is it not over
['hurrican'] 

Looking for someone but no name
['look', 'someon', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogan', '80', '90', 'destroy', 'hospit', 'st', 'croix', 'function', 'need', 'suppli', 'desper'] 

says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'haiti', 'rest', 'countri', 'today', 'tonight'] 



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [17]:

pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])



### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,shuffle=True)
#with active_session():
#     pipeline.fit(X_train,y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [19]:
target_names = y.columns
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))

NotFittedError: CountVectorizer - Vocabulary wasn't fitted.

### 6. Improve your model
Use grid search to find better parameters. 

In [20]:
pipeline.get_params()



parameters = {'clf__estimator__n_estimators': [300, 400]
             
             }

cv = GridSearchCV(pipeline, param_grid=parameters,cv=2, verbose=12, n_jobs=-1)


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [21]:
with active_session():
     cv.fit(X_train, y_train)

y_pred_cv = cv.predict(X_test)

target_names = y.columns


Fitting 2 folds for each of 2 candidates, totalling 4 fits
[CV] clf__estimator__n_estimators=300 ................................
[CV]  clf__estimator__n_estimators=300, score=0.23164218958611482, total= 7.4min
[CV] clf__estimator__n_estimators=300 ................................


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  8.8min remaining:    0.0s


[CV]  clf__estimator__n_estimators=300, score=0.2378409307648293, total= 7.5min
[CV] clf__estimator__n_estimators=400 ................................


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed: 17.5min remaining:    0.0s


[CV]  clf__estimator__n_estimators=400, score=0.22363150867823764, total= 9.2min
[CV] clf__estimator__n_estimators=400 ................................


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed: 28.0min remaining:    0.0s


[CV]  clf__estimator__n_estimators=400, score=0.23164218958611482, total= 9.3min


[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed: 38.7min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed: 38.7min finished


In [22]:
print(classification_report(y_test, y_pred_cv, target_names=target_names))

                        precision    recall  f1-score   support

               related       0.87      0.88      0.88      3938
               request       0.74      0.56      0.64       895
                 offer       0.00      0.00      0.00        26
           aid_related       0.75      0.66      0.70      2131
          medical_help       0.55      0.32      0.41       422
      medical_products       0.53      0.31      0.39       270
     search_and_rescue       0.37      0.17      0.24       127
              security       0.13      0.05      0.07        88
              military       0.49      0.34      0.40       155
           child_alone       0.00      0.00      0.00         0
                 water       0.73      0.65      0.69       339
                  food       0.82      0.69      0.75       595
               shelter       0.76      0.57      0.65       470
              clothing       0.65      0.44      0.52        73
                 money       0.45      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [25]:
with open('classifier.pkl', 'wb') as file:
    pickle.dump(cv, file)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [10]:

pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize,max_df=1.0,max_features=None,ngram_range=(1, 2))),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(SGDClassifier(loss="hinge",alpha=0.00005,tol=0.001,max_iter=1000)))])


In [11]:
parameters = {'clf__estimator__penalty': ['l1', 'l2']
             
             }

cv = GridSearchCV(pipeline, param_grid=parameters,cv=4, verbose=12, n_jobs=-1)

In [ ]:
with active_session():
     cv.fit(X_train,y_train)

Fitting 4 folds for each of 2 candidates, totalling 8 fits
[CV] clf__estimator__penalty=l1 ......................................


target_names = y.columns
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))

In [ ]:
target_names = y.columns
y_pred = cv.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))

### 9. Export your model as a pickle file

In [21]:


target_names = y.columns
report= classification_report(y_pred_cv,y_test, target_names=target_names)
    
temp=[]
          
for item in report.split("\n"):
    temp.append(item.strip().split('     '))
clean_list=[x for x in temp if x != ['']]
report_df=pd.DataFrame(clean_list[1:],columns=['group','precision','recall', 'f1-score','support'])
report_df

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


,group,precision,recall,f1-score,support
0,related,0.94,0.84,0.89,4406
1,request,0.48,0.85,0.62,512
2,offer,0.00,0.00,0.00,0
3,aid_related,0.71,0.76,0.73,1989
4,medical_help,0.09,0.72,0.16,53
5,medical_products,0.07,0.83,0.14,24
6,search_and_rescue,0.07,0.60,0.13,15
7,security,0.01,0.25,0.02,4
8,military,0.05,0.53,0.09,15
9,child_alone,0.00,0.00,0.00,0


In [ ]:
with open('classifier.pkl', 'wb') as file:
    pickle.dump(pipeline, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [91]:
for column in y.columns:
    print(column)
    print(y_test[column].value_counts())

related
1    3938
0    1306
Name: related, dtype: int64
request
0    4349
1     895
Name: request, dtype: int64
offer
0    5218
1      26
Name: offer, dtype: int64
aid_related
0    3113
1    2131
Name: aid_related, dtype: int64
medical_help
0    4822
1     422
Name: medical_help, dtype: int64
medical_products
0    4974
1     270
Name: medical_products, dtype: int64
search_and_rescue
0    5117
1     127
Name: search_and_rescue, dtype: int64
security
0    5156
1      88
Name: security, dtype: int64
military
0    5089
1     155
Name: military, dtype: int64
child_alone
0    5244
Name: child_alone, dtype: int64
water
0    4905
1     339
Name: water, dtype: int64
food
0    4649
1     595
Name: food, dtype: int64
shelter
0    4774
1     470
Name: shelter, dtype: int64
clothing
0    5171
1      73
Name: clothing, dtype: int64
money
0    5140
1     104
Name: money, dtype: int64
missing_people
0    5184
1      60
Name: missing_people, dtype: int64
refugees
0    5073
1     171
Name: refugees, dty